In [1]:
import torch
import matplotlib.pyplot as plt
import os
import struct
from collections import  defaultdict

from transformers import T5ForConditionalGeneration

In [2]:
def get_exponent(model, nbits):
    exponent = {}
    for idx, (name, param) in enumerate(model.named_parameters()):
        # assert param.dtype is torch.float16
        if param.ndim == 2 and param.shape[0] != 1 and param.shape[1] != 1:
            r, c = param.shape
            tensor_data = list(
                map(lambda x: int(
                        "{}".format(
                                bin(
                                    int.from_bytes(
                                    struct.pack('>e', x), byteorder='big'
                                    )
                                )[3: 3+nbits]      # torch.float16 [3: 8] torch.bfloat16[3: 11]
                            ), 
                            base=2
                    ),
                    param.abs().neg().reshape(-1).tolist()
                )
            )
            tensor = torch.tensor(tensor_data, dtype=torch.int8).reshape(r, c)
            exponent[name] = tensor
    return exponent

In [3]:
### Get the exponent ###
models_hub = {
    "t5": {
        "path": "/home/styaeng/project/delta-compress/pretrained_model/t5",       ### 这里要写成下载后的模型权重文件所在的路径
        "hdlr": T5ForConditionalGeneration.from_pretrained
    }
}
t5_model = models_hub['t5']['hdlr'](models_hub['t5']['path'])
t5_exponent = get_exponent(t5_model, nbits=5)

In [4]:
'''
2 bits
    [0 | 1 | 2 | 3]
3 bits
    [0 | 1 | 2 | 3 | 4 | 5 | 6 | 7]
4 bits
    [0-15]
5 bits
    [0]
'''

pattern = [
    [0, 1, 2, 3],
    [0, 1, 2, 3, 4, 5, 6, 7],
    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    [0],
]

In [5]:
### Get the percentage in tile granularity ###
tile_gran_pattern = []
PageSize = 4 * 1024 * 8 # bits
FP16 = 16   # bits
for k, tensor in t5_exponent.items():
    tile_pattern = defaultdict(dict)
    row, col = tensor.shape
    if col * FP16 <= PageSize:
        tile_pattern[k] = "LTP"     # a.k.a of "Less Than PageSize"
    else:
        for row_idx in range(row):
            tile = tensor[row_idx]
            for shift in range(3, -1, -1):
                tile_pattern[k][row_idx] = defaultdict(dict)
                idx = 3 - shift
                pat = pattern[idx]
                tmp = tile >> shift
                for elem in pat:
                    tile_pattern[k][row_idx][shift][elem] = torch.count_nonzero(tmp == elem).item()
    tile_gran_pattern.append(tile_pattern)

In [235]:
### Get the percentage in matrix granularity ###
PageSize = 4 * 1024 * 8 # bits
FP16 = 16   # bits
mat_gran_pattern = []
for name, tensor in t5_exponent.items():
    row, col = tensor.shape
    higher_cur = 0
    lower_cur = higher_cur + PageSize // (col * FP16)
    tensor_pattern = defaultdict(dict)
    tensor_pattern['name'] = name
    while lower_cur <= row:
        tile_pattern = {}
        tile = tensor[higher_cur: lower_cur]
        for shift in range(3, -1, -1):
            idx = 3 - shift
            pat = pattern[idx]
            tmp = tile >> shift
            tile_pattern[idx+2] = {}
            for elem in pat:
                tile_pattern[idx+2][elem] = torch.count_nonzero(tmp == elem).item()
        for bit in range(2, 5):
            values = torch.tensor(list(tile_pattern[bit].values())).sort()[0]
            values = torch.sort(values, descending=True)[0]
            wasted_bits = (tile.numel() * FP16 + PageSize - 1) // PageSize * PageSize - values.sum() * FP16
            flag = False
            compressed_count = 0
            for grp_count, v in enumerate(values):
                v = v.item()
                wasted_bits -= (v * (FP16 - bit) + bit - v * FP16)
                compressed_count += v
                original_bit_count = tile.numel() * FP16
                original_page_count = (original_bit_count + PageSize - 1) // PageSize
                compressed_bit_count = (v * (FP16 - bit) + bit) * grp_count + (values.sum().item() - compressed_count) * FP16
                compressed_page_count = (compressed_bit_count + PageSize - 1) // PageSize
                frac = compressed_page_count * PageSize - compressed_bit_count
                if wasted_bits >= PageSize:
                    flag = True
                    # print(f"Compress the tile successfully and free 1 page storage space!\
                    # \nParam_name: {k}\
                    # \nHigher_cur: {higher_cur}:{lower_cur}\
                    # \nOriginal_bit_count = {original_bit_count} bits\
                    # \nOriginal_page_count = {original_page_count} pages\
                    # \nCompressed_bit_count = {compressed_bit_count} bits\
                    # \nCompressed_page_count = {compressed_page_count} pages\
                    # \nBits={bit}, group_counts={grp_count}, wasted_bits={wasted_bits} ({(wasted_bits / PageSize):.2f} page), frac={frac / (compressed_page_count * PageSize) * 100:.2f} %")
                    break
        if lower_cur == row:
            tensor_pattern[f"{higher_cur}:{lower_cur}"]["original_bit_count"] = original_bit_count
            tensor_pattern[f"{higher_cur}:{lower_cur}"]["original_page_count"] = original_page_count
            tensor_pattern[f"{higher_cur}:{lower_cur}"]["compressed_bit_count"] = compressed_bit_count
            tensor_pattern[f"{higher_cur}:{lower_cur}"]["compressed_page_count"] = compressed_page_count
            tensor_pattern[f"{higher_cur}:{lower_cur}"]["compressed_ratio"] = (original_page_count - compressed_page_count) / original_page_count
            break
        if not flag:
            if (row - higher_cur) * col * FP16 < PageSize:
                lower_cur = row
            else:
                lower_cur += 1
        else:
            if frac > compressed_bit_count:
                if (row - higher_cur) * col * FP16 < PageSize:
                    lower_cur = row
                else:
                    lower_cur += 1
            else:
                tensor_pattern[f"{higher_cur}:{lower_cur}"]["original_bit_count"] = original_bit_count
                tensor_pattern[f"{higher_cur}:{lower_cur}"]["original_page_count"] = original_page_count
                tensor_pattern[f"{higher_cur}:{lower_cur}"]["compressed_bit_count"] = compressed_bit_count
                tensor_pattern[f"{higher_cur}:{lower_cur}"]["compressed_page_count"] = compressed_page_count
                tensor_pattern[f"{higher_cur}:{lower_cur}"]["compressed_ratio"] = (original_page_count - compressed_page_count) / original_page_count
                if (row - higher_cur) * col * FP16 < PageSize:
                    lower_cur = row
                else:
                    higher_cur = lower_cur
                    lower_cur = higher_cur + PageSize // (col * FP16)
                    if lower_cur > row:
                        lower_cur = row
    mat_gran_pattern.append(tensor_pattern)

In [246]:
# compressed_ratio = 0
# for idx, (k, v) in enumerate(t5_exponent.items()):
#     stats = list(mat_gran_pattern[idx].values())[1:]
#     for stat in stats:
#         cp_r = stat['compressed_ratio']
#         compressed_ratio += cp_r

In [165]:
for bit in range(2, 5):
    values = torch.tensor(list(tile_pattern[bit].values())).sort()[0]
    values = torch.sort(values, descending=True)[0]
    wasted_bits = factor * PageSize - values.sum() * FP16
    for grp_count, v in enumerate(values):
        wasted_bits -= (v * (FP16 - bit) + bit - v * FP16)
        # print(f"bit={bit}, group_counts={grp_count+1}, wasted_bits={wasted_bits} bits")
        if wasted_bits >= PageSize:
            print(f"Compress the tile successfully and free 1 page storage space!\
            \nWith bits={bit}, group_counts={grp_count}, wasted_bits={wasted_bits} ({(wasted_bits / PageSize):.2f} page)!")
            break

Compress the tile successfully and free 1 page storage space!            
With bits=4, group_counts=2, wasted_bits=38044 (1.16 page)!


In [46]:
### Get the percentage in matrix granularity ###
mat_gran_pattern = []
PageSize = 4 * 1024 * 8 # bits
FP16 = 16   # bits

for k, v in t5_exponent.items():
    row, col = v.shape
    higher_cur = 0
    lower_cur = higher_cur + PageSize // (col * FP16)
    while lower_cur <= row:
        """
        # 优化点:
            1. 没必要一行一行地进行计算，在初始tile存储占用还没有到PageSize的时候是没有必要计算压缩后的存储收益的；
        if tile[higher_cur]_compressed_size < PageSize:
            lower_cur = lower_cur + 1
        if tile[]
        """ 
        tile = v[higher_cur: lower_cur]
        

torch.Size([1, 512])
torch.Size([2, 512])
